In [1]:
!pwd

/content


## **Motivation**
<a name="Motivation"></a>

#### In such a fast-paced world where information on the internet is exploding, staying connecte, forming my own perspective, and staying sharp is incredibly challenging. Yet, with full passion for learning and growth, I’m always striving to improve.


<hr>        






## **Pre-requisition**

In [ ]:
%%capture
!pip install youtube_transcript_api elevenlabs langfun --pre

In [ ]:
# Let's add path of virtual environment site-packages to system path
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append("/content/drive/MyDrive/jing_env_colab/colab_env/lib/python3.10/site-packages")
sys.path.append('/content/drive/MyDrive/ColabNotebooks')  # Replace with the folder path


import langfun as lf
import pyglove as pg
from typing import Literal
import pandas as pd
import numpy as np
import os
import google.generativeai as genai
from google.colab import userdata
from langfun.core.structured import function_generation

Mounted at /content/drive


## **Build your crew to help you achieve your goal,  choose them wisely!**
<a name="Build your crew"></a>

In [ ]:
## Get the key and LLMs
gemini_key = userdata.get('gemini_key')
claude_key = userdata.get('claude_key')
openai_key = userdata.get('openai_key')
youtube_key = userdata.get('youtube_key')

## This is your crew, help to achieve your goal/tasks!! Choose them wisely, since each of them have different expertise.
lm_claude = lf.llms.Claude35Sonnet20241022(api_key=claude_key, temperature=0.6) ## good at coding
lm_openai = lf.llms.Gpt4o(api_key = openai_key, temperature=0.6) ## good at summarization

# **Task 1 - Help you Write the code to retrieval all the information from a Youtube Video**


In [ ]:
code = lf.query(prompt = "Help me to write a python code that achieve retreival the youtube video basic info, caption, and comments",
                # ==========================================================
                # Prompt SECTION: It's your request,  what you want LLM to do?
                # ==========================================================

                lm = lm_claude,
                # ==========================================================
                # lm SECTION: Your LLM Crew:  eg, Use Claude to write code
                # ==========================================================

                schema=lf.PythonCode,
                # ==========================================================
                # schema SECTION: Your desired output, eg, this output should Python code
                # ==========================================================
    )

In [ ]:
code

PythonCode(source='\nfrom googleapiclient.discovery import build\nfrom youtube_transcript_api import YouTubeTranscriptApi\n\n# Set up YouTube API client\nAPI_KEY = \'YOUR_API_KEY\'\nyoutube = build(\'youtube\', \'v3\', developerKey=API_KEY)\n\ndef get_video_info(video_id):\n    # Get video details\n    video_response = youtube.videos().list(\n        part=\'snippet,statistics\',\n        id=video_id\n    ).execute()\n    \n    video_info = video_response[\'items\'][0]\n    \n    # Get video captions\n    try:\n        captions = YouTubeTranscriptApi.get_transcript(video_id)\n    except:\n        captions = "No captions available"\n        \n    # Get video comments\n    comments = []\n    try:\n        comments_response = youtube.commentThreads().list(\n            part=\'snippet\',\n            videoId=video_id,\n            maxResults=100\n        ).execute()\n        \n        for item in comments_response[\'items\']:\n            comment = item[\'snippet\'][\'topLevelComment\'][\'snippet\']\n            comments.append({\n                \'author\': comment[\'authorDisplayName\'],\n                \'text\': comment[\'textDisplay\'],\n                \'likes\': comment[\'likeCount\'],\n                \'published_at\': comment[\'publishedAt\']\n            })\n    except:\n        comments = "Comments are disabled for this video"\n        \n    return {\n        \'title\': video_info[\'snippet\'][\'title\'],\n        \'description\': video_info[\'snippet\'][\'description\'], \n        \'view_count\': video_info[\'statistics\'][\'viewCount\'],\n        \'like_count\': video_info[\'statistics\'][\'likeCount\'],\n        \'captions\': captions,\n        \'comments\': comments\n    }\n\n# Example usage\nvideo_id = \'VIDEO_ID\'  # Extract from YouTube URL\nvideo_data = get_video_info(video_id)\nprint(video_data)\n')

In [ ]:
## Simply copy paste the code above,  and replace the API_KEY and Youtube_id you need to fetch!

from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi

# Replace with your API key
API_KEY = youtube_key

def get_video_info(video_id):
    # Create YouTube API client
    youtube = build('youtube', 'v3', developerKey=API_KEY)

    # Get video details
    video_response = youtube.videos().list(
        part='snippet,statistics',
        id=video_id
    ).execute()

    if not video_response['items']:
        return None

    video_info = video_response['items'][0]

    # Get video captions
    try:
        captions = YouTubeTranscriptApi.get_transcript(video_id)
    except:
        captions = None

    # Get video comments
    comments = []
    try:
        comments_response = youtube.commentThreads().list(
            part='snippet',
            videoId=video_id,
            maxResults=100
        ).execute()

        for item in comments_response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'author': comment['authorDisplayName'],
                'text': comment['textDisplay'],
                'likes': comment['likeCount'],
                'published_at': comment['publishedAt']
            })
    except:
        pass

    return {
        'title': video_info['snippet']['title'],
        'description': video_info['snippet']['description'],
        'view_count': video_info['statistics']['viewCount'],
        'like_count': video_info['statistics']['likeCount'],
        'comment_count': video_info['statistics']['commentCount'],
        'captions': captions,
        'comments': comments
    }



In [ ]:
# Example usage:
video_id = 'VMj-3S1tku0'  # Replace with actual video ID
video_data = get_video_info(video_id)

In [ ]:
video_data['captions'][2]

{'text': 'networks for a bit more than a decade',
 'start': 2.96,
 'duration': 3.839}

In [ ]:
video_data['comments'][10]

{'author': '@shreyaagarwal1123',
 'text': 'really thank u for making me understand  how neural network actually works makes me think how useless my uni actually is',
 'likes': 0,
 'published_at': '2024-11-24T10:35:48Z'}

# **Task 2 - Help you learn the key insights from video and comments,  and start your learning/insights/understanding**

In [ ]:
class MajorInsights(pg.Object):  ## Your output, you define it
  summary: str
  key_insights: list[str]
  questions: list[str]

major_insights = lf.query(prompt = "{{video_data}}", video_data=video_data, schema=MajorInsights, lm=lm_openai)

In [ ]:
major_insights

0key_insights[0],str'Micrograd is a minimalistic autograd engine that helps illustrate the principles of backpropagation.'Micrograd is a minimalistic autograd engine that helps illustrate the principles of backpropagation.
1key_insights[1],"str'Neural networks are mathematical expressions that can be broken down into fundamental operations like addition, multiplication, and non-linear activation functions.'Neural networks are mathematical expressions that can be broken down into fundamental operations like addition, multiplication, and non-linear activation functions."
2key_insights[2],"str'Backpropagation involves computing the gradients of the loss function with respect to each parameter in the network, allowing for optimization through gradient descent.'Backpropagation involves computing the gradients of the loss function with respect to each parameter in the network, allowing for optimization through gradient descent."
3key_insights[3],str'The video emphasizes the pedagogical value of understanding neural networks at a scalar level before moving to tensor-based operations used in production environments like PyTorch.'The video emphasizes the pedagogical value of understanding neural networks at a scalar level before moving to tensor-based operations used in production environments like PyTorch.
4key_insights[4],"str'Common pitfalls in implementing neural networks include not resetting gradients between iterations, which can lead to incorrect updates.'Common pitfalls in implementing neural networks include not resetting gradients between iterations, which can lead to incorrect updates."
0questions[0],str'How does micrograd compare to larger frameworks like PyTorch in terms of functionality and efficiency?'How does micrograd compare to larger frameworks like PyTorch in terms of functionality and efficiency?
1questions[1],str'What are the advantages and disadvantages of using scalar operations versus tensor operations in neural network training?'What are the advantages and disadvantages of using scalar operations versus tensor operations in neural network training?
2questions[2],str'How do different non-linear activation functions affect the learning and performance of neural networks?'How do different non-linear activation functions affect the learning and performance of neural networks?
3questions[3],str'Can micrograd be extended to support more complex neural network architectures or larger datasets?'Can micrograd be extended to support more complex neural network architectures or larger datasets?
4questions[4],str'What are the best practices for selecting learning rates and managing gradient updates to ensure stable training?'What are the best practices for selecting learning rates and managing gradient updates to ensure stable training?
